In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
X = np.random.uniform(low=0.0, high=3.0, size=(1000, 2))

In [ ]:
y_input = X[:, 0] * X[:, 1]
filter_ = y_input < 1.0
y_input[filter_] = 0
y_input[~filter_] = 1

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_input)
plt.show()

In [ ]:
x_input = np.zeros((1000, 3))
x_input[:, 0:2] = X
x_input[:, 2] = X[:, 0]*X[:, 1]
x_input = x_input.astype(np.float32)
y_input = y_input.reshape((1000, 1)).astype(np.float32)

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=(None, 3), name='input')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='output')

In [ ]:
W = tf.get_variable(name='w', shape=(3, 1), initializer=tf.random_normal_initializer(seed=0))
b = tf.get_variable(name='b', shape=(1, 1), initializer=tf.random_normal_initializer(seed=0))

In [ ]:
y_pred = tf.sigmoid(tf.matmul(x, W) + b)

In [ ]:
loss = tf.reduce_mean(-y*tf.log(y_pred) + (y-1)*tf.log(1-y_pred))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

In [ ]:
def batchify(x, y, size=100):
    counter = 0
    while counter < x.shape[0]:
        yield x[counter:counter+size, :], y[counter:counter+size]
        counter += size

In [ ]:
EPOCH = 2300
BATCH_SIZE = 100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(EPOCH):
        for x_batch, y_batch in batchify(x_input, y_input):
            pred_vals, loss_val, _ = sess.run([y_pred, loss, optimizer], feed_dict={x: x_batch, y:y_batch})
        if epoch % 100 == 0:
            pred_vals[pred_vals < 0.5] = 0
            pred_vals[pred_vals > 0.5] = 1
            print("epoch {}, loss {}, batch accuracy {}".format(epoch, loss_val,
                                                          accuracy_score(np.squeeze(y_batch), 
                                                                         np.squeeze(pred_vals))))
    print("final loss {}". format(loss_val))
    pred_vals = sess.run(y_pred, feed_dict={x: x_input, y:y_input})
    print("final weights: \n", W.eval())
    print("final bias: \n", b.eval())

In [ ]:
pred_vals = np.squeeze(pred_vals)
plt.scatter(X[:, 0], X[:, 1], c = pred_vals)
plt.colorbar()
plt.show()